In [2]:
# 1_setup_analytical_db.ipynb

import pandas as pd
import numpy as np
import logging
from data201 import db_connection

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Connect to the database
conn = db_connection(config_file='premier_league_analytics.ini')
cursor = conn.cursor()

# Execute a test query
cursor.execute("SELECT COUNT(*) FROM Teams")
team_count = cursor.fetchone()[0]
print(f"Connected to database with {team_count} teams")

# Create analytical database schema
# Start with the simplest tables first

# 1. Create dim_Time
print("Creating Time dimension...")
cursor.execute("""
CREATE TABLE IF NOT EXISTS `dim_Time` (
  `TimeID` INT AUTO_INCREMENT PRIMARY KEY,
  `Date` DATE NOT NULL,
  `DayOfWeek` INT,
  `DayName` VARCHAR(10),
  `DayOfMonth` INT,
  `DayOfYear` INT,
  `WeekOfYear` INT,
  `Month` INT,
  `MonthName` VARCHAR(10),
  `Quarter` INT,
  `Year` INT,
  `Season` VARCHAR(10),
  `Matchday` INT,
  `IsWeekend` BOOLEAN,
  `IsHoliday` BOOLEAN,
  INDEX `idx_time_date` (`Date`)
)
""")

# 2. Create fact_MatchResult
print("Creating Match Result fact table...")
cursor.execute("""
CREATE TABLE IF NOT EXISTS `fact_MatchResult` (
  `MatchResultID` INT AUTO_INCREMENT PRIMARY KEY,
  `MatchID` INT NOT NULL,
  `TimeID` INT NOT NULL,
  `HomeTeamID` INT NOT NULL,
  `AwayTeamID` INT NOT NULL,
  `RefereeID` INT,
  `SeasonID` INT NOT NULL,
  `DivisionID` INT NOT NULL,
  `HomeGoals` INT NOT NULL,
  `AwayGoals` INT NOT NULL,
  `Result` CHAR(1) NOT NULL,
  `HalfTimeHomeGoals` INT,
  `HalfTimeAwayGoals` INT,
  `HalfTimeResult` CHAR(1),
  `HomePoints` INT GENERATED ALWAYS AS (CASE WHEN Result = 'H' THEN 3 WHEN Result = 'D' THEN 1 ELSE 0 END) STORED,
  `AwayPoints` INT GENERATED ALWAYS AS (CASE WHEN Result = 'A' THEN 3 WHEN Result = 'D' THEN 1 ELSE 0 END) STORED,
  `ETLBatchID` INT DEFAULT 1,
  `LoadTimestamp` DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (`TimeID`) REFERENCES `dim_Time`(`TimeID`),
  INDEX `idx_match_time` (`TimeID`),
  INDEX `idx_match_season` (`SeasonID`),
  INDEX `idx_match_teams` (`HomeTeamID`, `AwayTeamID`)
)
""")

# 3. Create fact_TeamMatchStats
print("Creating Team Match Stats fact table...")
cursor.execute("""
CREATE TABLE IF NOT EXISTS `fact_TeamMatchStats` (
  `TeamStatsID` INT AUTO_INCREMENT PRIMARY KEY,
  `MatchID` INT NOT NULL,
  `TeamID` INT NOT NULL,
  `TimeID` INT NOT NULL,
  `SeasonID` INT NOT NULL,
  `IsHomeTeam` BOOLEAN NOT NULL,
  `OpponentID` INT NOT NULL,
  `Goals` INT NOT NULL,
  `GoalsConceded` INT NOT NULL,
  `Shots` INT,
  `ShotsOnTarget` INT,
  `Corners` INT,
  `Fouls` INT,
  `YellowCards` INT,
  `RedCards` INT,
  `Result` CHAR(1),
  `Points` INT,
  `ETLBatchID` INT DEFAULT 1,
  `LoadTimestamp` DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (`TimeID`) REFERENCES `dim_Time`(`TimeID`),
  INDEX `idx_teamstats_team` (`TeamID`),
  INDEX `idx_teamstats_match` (`MatchID`),
  INDEX `idx_teamstats_time` (`TimeID`)
)
""")

# 4. Create fact_LeagueSnapshot
print("Creating League Snapshot fact table...")
cursor.execute("""
CREATE TABLE IF NOT EXISTS `fact_LeagueSnapshot` (
  `SnapshotID` INT AUTO_INCREMENT PRIMARY KEY,
  `SeasonID` INT NOT NULL,
  `DivisionID` INT NOT NULL,
  `TimeID` INT NOT NULL,
  `TeamID` INT NOT NULL,
  `Position` INT NOT NULL,
  `MatchesPlayed` INT NOT NULL,
  `Won` INT NOT NULL,
  `Drawn` INT NOT NULL,
  `Lost` INT NOT NULL,
  `GoalsFor` INT NOT NULL,
  `GoalsAgainst` INT NOT NULL,
  `GoalDifference` INT GENERATED ALWAYS AS (GoalsFor - GoalsAgainst) STORED,
  `Points` INT NOT NULL,
  `Form` VARCHAR(10),
  `LastMatchID` INT,
  `ETLBatchID` INT DEFAULT 1,
  `LoadTimestamp` DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (`TimeID`) REFERENCES `dim_Time`(`TimeID`),
  INDEX `idx_snapshot_season` (`SeasonID`, `DivisionID`),
  INDEX `idx_snapshot_team` (`TeamID`)
)
""")

# Create analytical views
print("Creating analytical views...")
cursor.execute("""
CREATE OR REPLACE VIEW vw_LeagueTable AS
SELECT 
    s.SeasonName,
    d.LeagueName,
    t.TeamName,
    ls.Position,
    ls.MatchesPlayed,
    ls.Won,
    ls.Drawn, 
    ls.Lost,
    ls.GoalsFor,
    ls.GoalsAgainst,
    ls.GoalDifference,
    ls.Points,
    ROUND(ls.Points * 1.0 / ls.MatchesPlayed, 2) AS PointsPerGame,
    ls.Form
FROM fact_LeagueSnapshot ls
JOIN Seasons s ON ls.SeasonID = s.SeasonID
JOIN Divisions d ON ls.DivisionID = d.DivisionID
JOIN Teams t ON ls.TeamID = t.TeamID
JOIN dim_Time dt ON ls.TimeID = dt.TimeID
WHERE ls.TimeID = (
    SELECT MAX(TimeID) 
    FROM fact_LeagueSnapshot 
    WHERE SeasonID = ls.SeasonID
)
ORDER BY ls.SeasonID, ls.Position
""")

cursor.execute("""
CREATE OR REPLACE VIEW vw_HomeAdvantage AS
SELECT 
    s.SeasonName,
    t.TeamName,
    SUM(CASE WHEN tms.IsHomeTeam = 1 THEN 1 ELSE 0 END) AS HomeMatches,
    SUM(CASE WHEN tms.IsHomeTeam = 0 THEN 1 ELSE 0 END) AS AwayMatches,
    ROUND(AVG(CASE WHEN tms.IsHomeTeam = 1 THEN tms.Goals ELSE NULL END), 2) AS AvgHomeGoalsScored,
    ROUND(AVG(CASE WHEN tms.IsHomeTeam = 0 THEN tms.Goals ELSE NULL END), 2) AS AvgAwayGoalsScored,
    ROUND(AVG(CASE WHEN tms.IsHomeTeam = 1 THEN tms.Points ELSE NULL END), 2) AS AvgHomePoints,
    ROUND(AVG(CASE WHEN tms.IsHomeTeam = 0 THEN tms.Points ELSE NULL END), 2) AS AvgAwayPoints,
    ROUND(AVG(CASE WHEN tms.IsHomeTeam = 1 THEN tms.Points ELSE NULL END) - 
          AVG(CASE WHEN tms.IsHomeTeam = 0 THEN tms.Points ELSE NULL END), 2) AS HomeAdvantagePoints
FROM fact_TeamMatchStats tms
JOIN Teams t ON tms.TeamID = t.TeamID
JOIN Matches m ON tms.MatchID = m.MatchID
JOIN Seasons s ON tms.SeasonID = s.SeasonID
GROUP BY s.SeasonName, t.TeamName
HAVING COUNT(CASE WHEN tms.IsHomeTeam = 1 THEN 1 END) > 5 
   AND COUNT(CASE WHEN tms.IsHomeTeam = 0 THEN 1 END) > 5
ORDER BY HomeAdvantagePoints DESC
""")

conn.commit()
print("Analytical database schema created successfully!")

# Close connection
cursor.close()
conn.close()

Connected to database with 20 teams
Creating Time dimension...
Creating Match Result fact table...
Creating Team Match Stats fact table...
Creating League Snapshot fact table...
Creating analytical views...
Analytical database schema created successfully!
